In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import dlib
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt

In [2]:
model = load_model('deepfake-detection-model.h5')

In [ ]:
input_shape = (128, 128, 3)
pr_data = []
croped_images = []
detector = dlib.get_frontal_face_detector()
cap = cv2.VideoCapture('v2.mp4')
ret,frame = cap.read()

frameRate = cap.get(1)
k=1
while cap.isOpened():

    frameId = cap.get(1)
    ret, frame = cap.read()
    if ret != True:
        break
    if frameId % ((int(frameRate)+1)*1) == 0:
        face_rects, scores, idx = detector.run(frame, 0)
        for i, d in enumerate(face_rects):
            x1 = d.left()
            y1 = d.top()
            x2 = d.right()
            y2 = d.bottom()
            crop_img = frame[y1:y2, x1:x2]
            if(len(crop_img)==0):
                continue
            croped_images.append(crop_img)
            data = img_to_array(cv2.resize(crop_img, (128, 128))).flatten() / 255.0
            data = data.reshape(-1, 128, 128, 3)
            pr_data.append(int(model.predict_classes(data)))
#             print(model.predict_classes(data))

print(list(pr_data))
print(len(pr_data))

In [ ]:
import math as m
axes=[]
fig=plt.figure(figsize=(18,18))
r=np.sqrt(len(croped_images))+1
c = m.ceil(len(croped_images)/r)+1
for i in range(0, len(croped_images)):
    axes.append(fig.add_subplot(r, c, i+1))
    if pr_data[i]==0:
        t="fake"
    else:
        t="real"
    plt.title(t)
    img = cv2.cvtColor(croped_images[i], cv2.COLOR_BGR2RGB)
    plt.imshow(img, aspect='auto')
    video.write(frame)
plt.show()

In [ ]:
print("Number of extracted frames:", len(pr_data))
print("Probability that video is from real class:",np.mean(pr_data))